<a href="https://colab.research.google.com/github/kessingtonosazee/GCP_Project_1/blob/master/UCS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:




# Import necessary libraries
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
import tracemalloc
import time

# Set the visualization parameters
show_animation = True
pause_time = 0.1

# Define the UCS class for path planning
class UCS:

    def __init__(self, ox, oy, resolution, robot_radius, model='8n'):
        """
        Initialize the UCS path planner.

        Parameters:
        - ox: x position list of obstacles [m]
        - oy: y position list of obstacles [m]
        - resolution: grid resolution [m]
        - robot_radius: robot radius [m]
        - model: motion model type ('4n' or '8n')
        """
        # Initialize variables for map dimensions and obstacle representation
        self.min_x = None
        self.min_y = None
        self.max_x = None
        self.max_y = None
        self.x_width = None
        self.y_width = None
        self.obstacle_map = None

        # Set parameters for the map
        self.resolution = resolution
        self.robot_radius = robot_radius

        # Calculate obstacle map based on provided obstacle positions
        self.calc_obstacle_map(ox, oy)

        # Choose a motion model based on the provided type
        if model == '4n':
            self.motion = self.get_motion_model_4n()
        else:
            self.motion = self.get_motion_model_8n()   # This is the eight directional motion model

    # Nested class definition for a Node in the search space
    class Node:
        def __init__(self, x, y, cost, parent_index):
            self.x = x  # index of grid
            self.y = y  # index of grid
            self.cost = cost
            self.parent_index = parent_index  # index of the previous Node

        def __str__(self):
            return str(self.x) + "," + str(self.y) + "," + str(
                self.cost) + "," + str(self.parent_index)

    def planning(self, sx, sy, gx, gy):
        """
        Perform UCS path search.

        Parameters:
        - sx: start x position [m]
        - sy: start y position [m]
        - gx: goal x position [m]
        - gy: goal y position [m]

        Returns:
        - rx: x position list of the final path
        - ry: y position list of the final path
        """
        # Create start and goal nodes
        start_node = self.Node(self.calc_xy_index(sx, self.min_x),
                               self.calc_xy_index(sy, self.min_y), 0.0, -1)
        goal_node = self.Node(self.calc_xy_index(gx, self.min_x),
                              self.calc_xy_index(gy, self.min_y), 0.0, -1)

        # Visualization: Plot start and goal nodes
        plt.scatter(start_node.x, start_node.y, s=400, c='red', marker='s')
        plt.scatter(goal_node.x, goal_node.y, s=400, c='green', marker='s')

        # Initialize open set and closed set for path exploration
        open_set, closed_set = dict(), dict()
        open_set[self.calc_grid_index(start_node)] = start_node

        # Main UCS path planning loop
        while 1:
            c_id = min(open_set, key=lambda o: open_set[o].cost)
            current = open_set[c_id]

            # Visualization: Plot current node for animation
            if show_animation:
                plt.plot(self.calc_grid_position(current.x, self.min_x),
                         self.calc_grid_position(current.y, self.min_y), "xc")
                plt.scatter(self.calc_grid_position(current.x, self.min_x),
                            self.calc_grid_position(current.y, self.min_y),
                            s=300, c='yellow', marker='s')
                plt.pause(pause_time)

                # For stopping simulation with the escape key
                plt.gcf().canvas.mpl_connect(
                    'key_release_event',
                    lambda event: [exit(0) if event.key == 'escape' else None])
                if len(closed_set.keys()) % 10 == 0:
                    plt.pause(0.001)

            # Goal test: Check if the current node is the goal
            if current.x == goal_node.x and current.y == goal_node.y:
                print("Find goal")
                plt.scatter(current.x,
                            current.y,
                            s=100, c='green', marker='s')
                plt.pause(pause_time)

                # Generate the final path and visualize it
                locx, locy = self.calc_final_path(current, closed_set)
                if len(locx) > 2:
                    for i in range(len(locx) - 1):
                        px = (locx[i], locx[i + 1])
                        py = (locy[i], locy[i + 1])
                        plt.plot(px, py, "-m", linewidth=4)
                        plt.pause(pause_time)

                goal_node.parent_index = current.parent_index
                goal_node.cost = current.cost
                print("cost:", goal_node.cost)
                break

            # Remove the item from the open set
            del open_set[c_id]

            # Add it to the closed set
            closed_set[c_id] = current

            # Visualization: Plot current node in red to indicate it's visited
            plt.scatter(current.x,
                        current.y,
                        s=100, c='red', marker='s')
            plt
